In [1]:
import pandas as pd
import numpy as np
from anndata import AnnData
import anndata as ad
import scanpy as sc
import scipy.sparse as sp
import glob
from tqdm import tqdm
from paste import pairwise_align
import ot
import torch
import matplotlib.pyplot as plt

In [2]:
l = torch.nn.Linear(3,3).to('cuda:0')

# 切片的z坐标

In [3]:
z_csv = pd.read_csv('/home/hanyuji/Data/ST_data/macaque_cortex/mmc3_macaque.csv')
z_monkey_1 = z_csv[z_csv['Monkey'] == '#1']
z_monkey_1['z'] = pd.to_numeric(z_monkey_1['EBZ (mm)'].str.replace(' ', ''), errors='coerce')  # 字符串转换为浮点数

z_monkey_1['gap'] = z_monkey_1['z'].diff()
z_monkey_1.loc[z_monkey_1.index[0], 'gap'] = 0 # 将第一行的'gap'值设为0
z_monkey_1

/tmp/ipykernel_985340/2183458095.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z_monkey_1['z'] = pd.to_numeric(z_monkey_1['EBZ (mm)'].str.replace(' ', ''), errors='coerce')  # 字符串转换为浮点数
/tmp/ipykernel_985340/2183458095.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z_monkey_1['gap'] = z_monkey_1['z'].diff()


,Section ID,Monkey,EBZ (mm),Number of cells,z,gap
0,T155,#1,41.08,17613,41.08,0.00
1,T153,#1,40.06,42104,40.06,-1.02
2,T151,#1,39.58,53616,39.58,-0.48
3,T149,#1,39.05,50213,39.05,-0.53
4,T147,#1,38.56,37766,38.56,-0.49
...,...,...,...,...,...,...
114,T136,#1,-17.87,134776,-17.87,-0.63
115,T134,#1,-18.46,259903,-18.46,-0.59
116,T132,#1,-19.17,227914,-19.17,-0.71
117,T128,#1,-19.58,117120,-19.58,-0.41


In [4]:
# 切片的文件名index
file_index = z_monkey_1['Section ID'].to_list()
file_index

['T155',
 'T153',
 'T151',
 'T149',
 'T147',
 'T144',
 'T142',
 'T140',
 'T138',
 'T143',
 'T141',
 'T139',
 'T137',
 'T135',
 'T133',
 'T130',
 'T131',
 'T129',
 'T127',
 'T125',
 'T123',
 'T121',
 'T119',
 'T117',
 'T115',
 'T113',
 'T111',
 'T109',
 'T107',
 'T105',
 'T103',
 'T101',
 'T99',
 'T97',
 'T95',
 'T93',
 'T91',
 'T89',
 'T87',
 'T85',
 'T83',
 'T81',
 'T79',
 'T77',
 'T75',
 'T73',
 'T71',
 'T67',
 'T65',
 'T63',
 'T61',
 'T59',
 'T57',
 'T55',
 'T53',
 'T51',
 'T49',
 'T47',
 'T45',
 'T43',
 'T41',
 'T39',
 'T37',
 'T33',
 'T31',
 'T29',
 'T27',
 'T25',
 'T28',
 'T30',
 'T32',
 'T34',
 'T36',
 'T38',
 'T40',
 'T42',
 'T44',
 'T48',
 'T50',
 'T52',
 'T54',
 'T56',
 'T58',
 'T60',
 'T62',
 'T64',
 'T66',
 'T68',
 'T72',
 'T74',
 'T76',
 'T78',
 'T80',
 'T82',
 'T84',
 'T88',
 'T90',
 'T92',
 'T94',
 'T96',
 'T98',
 'T100',
 'T102',
 'T104',
 'T106',
 'T108',
 'T110',
 'T112',
 'T114',
 'T116',
 'T118',
 'T120',
 'T122',
 'T124',
 'T136',
 'T134',
 'T132',
 'T128',
 'T126'

In [7]:
value = z_monkey_1.loc[z_monkey_1['Section ID'] == 'T153', 'z'].values[0]
value

40.06

# 取切片的子集

In [16]:
def random_subset(adata, cell_num=5000, replace=False):
    '''
    随机选取 5000 个细胞
    '''
    
    random_indices = np.random.choice(adata.n_obs, cell_num, replace=replace)   # 从 276593 个细胞中随机选取 5000 个细胞的索引
    adata_subset = adata[random_indices, :]   # 使用选取的索引创建新的 AnnData 对象

    return adata_subset

# slice_0_subset = random_subset(slice_0)

# 加载数据

In [5]:
h5ad_dic = '/home/hanyuji/Data/ST_data/macaque_cortex/h5ad/'
# files = 'total_gene_T25.type20230503-macaque1-contour2.h5ad'
# file_index
# data_index = [153, 155]

adatas = []
for i, index in tqdm(enumerate(file_index)):
    data_path = h5ad_dic + f'total_gene_{index}.type20230503-macaque1-contour2.h5ad'
    adata = sc.read_h5ad(data_path)
    
    # 简单的预处理
    # adata.obs['slice_index'] = np.repeat(index, adata.n_obs)  # 添加切片序号信息
    adata.obsm['spatial_raw'] = adata.obsm['spatial'].copy()  # 复制obsm['spatial']中的数据
    adata.obsm['spatial'] = np.delete(adata.obsm['spatial'], [0, 1], axis=1)  # 删除obsm['spatial']中的前两列
    
    z_value = z_monkey_1.loc[z_monkey_1['Section ID'] == index, 'z'].values[0]
    gap_value = z_monkey_1.loc[z_monkey_1['Section ID'] == index, 'gap'].values[0]
    
    adata.obs['z'] = np.repeat(z_value, adata.n_obs)  # 添加z坐标
    adata.obs['gap'] = np.repeat(gap_value, adata.n_obs)  # 添加和上一张切片的gap
    adata.obs['slice'] = np.repeat(i, adata.n_obs)  # 添加切片顺序
    

    adata_subset = random_subset(adata,1000)
        
    adatas.append(adata_subset)

100%|██████████| 119/119 [02:31<00:00,  1.27s/it]


In [6]:
# 合并切片adata

combined_adata = ad.concat(
    adatas, 
    axis=0, 
    join='outer', 
    merge='unique', 
    uns_merge='unique', 
    label='batch', 
    keys=list(range(len(adatas))), 
    index_unique='-', 
    fill_value=0
)
combined_adata

AnnData object with n_obs × n_vars = 1428000 × 15929
    obs: 'chip', 'gene_area', 'slice_index', 'batch'
    obsm: 'spatial', 'spatial_raw'

# 计算高可变基因

In [7]:
def get_slice_HVG(slice, n_top_genes=2000, need_norm_and_log1p=False, target_sum=1e4):
    '''
    计算并选择前2000个高可变基因
    '''
    
    slice.var_names_make_unique()  # 确保变量名唯一，避免重复
    sc.pp.filter_genes(slice, min_counts=5)  # 过滤掉计数小于5的基因
    sc.pp.highly_variable_genes(slice, n_top_genes=n_top_genes, flavor='seurat')  # 使用seurat方法计算高可变基因，并选择前2000个
    slice = slice[:, slice.var.highly_variable]  # 只保留高可变基因
    
    if need_norm_and_log1p:
        sc.pp.normalize_total(slice, target_sum=target_sum)  # 将每个细胞的总表达量归一化为10000
        sc.pp.log1p(slice)  # 对数据进行对数变换
    
    return slice  # 返回处理后的数据


adatas_HVG = get_slice_HVG(combined_adata)

In [8]:
file_path = '/home/hanyuji/Workbench/ST/ST_data_check/paste_alignment_cortex/PASTE_align/cortex_macaque1_119slice_subset_1200spot_2000gene_HVG.h5ad'
adatas_HVG.write(file_path)

# 5000 subset

In [4]:
dataset_HVG_path = '/home/hanyuji/Workbench/ST/ST_data_check/paste_alignment_cortex/PASTE_align/cortex_macaque1_119slice_subset_12000spot_2000gene_HVG.h5ad'

adata = sc.read_h5ad(dataset_HVG_path)

In [5]:
# 根据batch值分开成多个AnnData对象
unique_batches = adata.obs['batch'].unique()

adata_list = [random_subset(adata[adata.obs['batch'] == batch].copy(),cell_num=1000) for batch in unique_batches]
# adata_list = [adata[adata.obs['batch'] == batch].copy() for batch in unique_batches]

# 保存切片subset
subset_combined_adata = ad.concat(
    adata_list, 
    axis=0, 
    join='outer', 
    merge='unique', 
    uns_merge='unique', 
    fill_value=0
)

dataset_HVG_subset_path = '/home/hanyuji/Workbench/ST/ST_data_check/paste_alignment_cortex/PASTE_align/cortex_macaque1_119slice_subset_1000spot_2000gene_HVG.h5ad'
subset_combined_adata.write(dataset_HVG_subset_path)